In [28]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
%autoreload 2

In [30]:
%matplotlib inline
from pathlib import Path
import functools
import itertools
from dataclasses import dataclass, fields

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams 

from matplottoy.data import fruit
from matplottoy.encoders import color, position
from matplottoy.artists import point, bar, utils


spath = Path("../../../draft/figures/code/")
rcParams['figure.dpi'] = 200

In [31]:
data = {'fruit': ['apple', 'orange', 'lemon', 'lime'], 
       'calories':[95, 67, 17, 20], 
       'juice':[True, True, False, False]}
df = pd.DataFrame(data)

In [39]:
fruit.fiberbundle(fruit.Section(df))('calories')(None)()

0    95
1    67
2    17
3    20
Name: calories, dtype: int64

In [ ]:
data = fruit.fiberbundle(fruit.Section(df))


In [33]:
aes = {'position': ['fruit'],
       'length': ['calories'],
       'floor': None, 
       'width': None,
       'facecolor': ['juice'],
       'edgecolor': None,
       'linewidth': None,
       'linestyle': None}

In [34]:
#rewrite nu __call__ to be  ufuncs
# rewrite Q as compute graph that doesn't get executed in draw
# pull up projection function (nu, projection function)
# remove placeholder arguments 
nus = {'position': lambda x : x, 
       'length' : position.Nominal({'apple': 0, 'orange': 2, 'lemon': 4, 'lime': 6}),
       'floor': lambda _: 0,
       'width': lambda _ : .8,
       'facecolor' : color.Nominal({True: 'orange', False: 'blue'}), 
       'edgecolor' : lambda _ : 'k', 
       'linewidth' :  lambda _ : 2,
       'linestyle' : lambda _ : (None, None)}

In [36]:
fruit.fiberbundle(df)(None)('juice')()

AttributeError: 'DataFrame' object has no attribute 'section'